In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,52563
2,Huelva,Confirmados PDIA 14 días,4056
3,Huelva,Tasa PDIA 14 días,"790,3813551064949"
4,Huelva,Confirmados PDIA 7 días,2350
5,Huelva,Tasa PDIA 7 dias,"457,93791531071577"
6,Huelva,Total Confirmados,52779
7,Huelva,Curados,47181
8,Huelva,Fallecidos,426


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  52563.0


/tmp/ipykernel_93852/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  14927.0


/tmp/ipykernel_93852/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_93852/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_93852/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_93852/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 166 personas en los últimos 7 días 

Un positivo PCR cada 101 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,52563.0,2350.0,4056.0,513170.0,457.937915,790.381355,1550.0
Huelva-Costa,31118.0,1528.0,2481.0,289548.0,527.719066,856.852750,1017.0
Huelva (capital),14927.0,868.0,1419.0,143837.0,603.460862,986.533368,569.0
Condado-Campiña,16051.0,491.0,926.0,156231.0,314.278216,592.712074,323.0
Sierra de Huelva-Andévalo Central,4959.0,327.0,643.0,67391.0,485.227998,954.133341,208.0
Ayamonte,2349.0,145.0,213.0,21104.0,687.073541,1009.287339,90.0
Aljaraque,1957.0,128.0,207.0,21474.0,596.069666,963.956412,80.0
Punta Umbría,1502.0,92.0,151.0,15355.0,599.153370,983.393032,67.0
Valverde del Camino,1027.0,81.0,102.0,12750.0,635.294118,800.000000,62.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
San Silvestre de Guzmán,59.0,33.0,34.0,614.0,5374.592834,5537.459283,29.0
Puerto Moral,21.0,5.0,11.0,278.0,1798.561151,3956.834532,2.0
Aracena,932.0,81.0,218.0,8255.0,981.223501,2640.823743,48.0
Fuenteheridos,34.0,13.0,18.0,709.0,1833.568406,2538.787024,5.0
Cumbres Mayores,195.0,16.0,44.0,1749.0,914.808462,2515.723270,7.0
Marines (Los),22.0,1.0,7.0,399.0,250.626566,1754.385965,0.0
Corteconcepción,57.0,2.0,7.0,536.0,373.134328,1305.970149,1.0
Villanueva de las Cruces,27.0,1.0,5.0,387.0,258.397933,1291.989664,0.0
Cortegana,368.0,28.0,59.0,4602.0,608.431117,1282.051282,21.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Marines (Los),22.0,1.0,7.0,399.0,250.626566,1754.385965,0.0,0.142857
Villanueva de las Cruces,27.0,1.0,5.0,387.0,258.397933,1291.989664,0.0,0.200000
Almendro (El),64.0,2.0,7.0,840.0,238.095238,833.333333,0.0,0.285714
Corteconcepción,57.0,2.0,7.0,536.0,373.134328,1305.970149,1.0,0.285714
Campillo (El),121.0,7.0,20.0,2024.0,345.849802,988.142292,3.0,0.350000
Cumbres Mayores,195.0,16.0,44.0,1749.0,914.808462,2515.723270,7.0,0.363636
Galaroza,67.0,4.0,11.0,1382.0,289.435601,795.947902,3.0,0.363636
Aracena,932.0,81.0,218.0,8255.0,981.223501,2640.823743,48.0,0.371560
Bollullos Par del Condado,1540.0,50.0,127.0,14387.0,347.535970,882.741364,27.0,0.393701
